In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import umap # dimensionality reduction
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import hdbscan
from deep_translator import GoogleTranslator

c:\anaconda\envs\machine_learning\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\anaconda\envs\machine_learning\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\anaconda\envs\machine_learning\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Preparing Dataset

In [2]:
dataset=pd.read_csv("C:/Users/rbw19/Downloads/dataset.csv",on_bad_lines='skip')
dataset=dataset[["title",'category_id','contents',"agency_id",'tweet_text']]
dataset.head(5)

,title,category_id,contents,agency_id,tweet_text
0,سمو ولي العهد يغادر مصر بعد المشاركة في مؤتمري...,أخبار محلية,\n بحفظ الل...,الوطن,NaN
1,الأمانة العامة للمجلس الأعلى للتخطيط والتنمية ...,أخبار إقتصادية,\r\n ...,الأمانة العامة للمجلس الأعلى للتخطيط والتنمية,NaN
2,"\r\n ""التأم...",أخبار محلية,\n\n\n\n\n\n \n\n\n,المؤسسة العامة للتأمينات الاجتماعية,NaN
3,تمديد طرح الممارسة رقم ص و م م 4/ 2022/2023م ...,أخبار إقتصادية,تمديد طرح الممارسة رقم ص و م م 4/ 2022/2023م ...,الصندوق الوطني,NaN
4,بسم اللّه الرحمن الرحيم,أخبار محلية,,جهاز متابعة الأداء الحكومي,NaN


In [3]:
dataset.fillna("",inplace=True)

In [4]:
exp='[\u0627-\u064aA-Za-z]+' #re to get arabic and english , i had eliminated numbers and special characters because they dont have much influence 
dataset["title"]=dataset["title"].apply(lambda data : " ".join(re.findall(exp,data)))
dataset['category_id']=dataset["category_id"].apply(lambda data: " ".join(re.findall(exp,data)) )
dataset["contents"]=dataset['contents'].apply(lambda data : " ".join(re.findall(exp,data)))
dataset["agency_id"]=dataset["agency_id"].apply(lambda data : " ".join(re.findall(exp,data)))
dataset["tweet_text"]=dataset["tweet_text"].apply(lambda data: " ".join(re.findall(exp,data)))


In [ ]:
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    # text = text.lower()
    # text = text.replace('x', '')
    return text

dataset["title"]=dataset["title"].apply(lambda data:cleanText(data))
dataset["contents"]=dataset["contents"].apply(lambda data:cleanText(data))
dataset["category_id"]=dataset["category_id"].apply(lambda data:cleanText(data))
dataset["agency_id"]=dataset["agency_id"].apply(lambda data:cleanText(data))
dataset["tweet_text"]=dataset["tweet_text"].apply(lambda data:cleanText(data))

In [ ]:
data=dataset["title"]+dataset["contents"]+dataset["category_id"]+dataset["agency_id"]+dataset["tweet_text"]
data=data.to_list()

# Vectorizing

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" # this is already a pretrained model ,so need of saving this model with our data
model = hub.load(module_url)

In [ ]:
embedding_gen = model(data)  #

In [ ]:
embedding_gen.shape

# Dimensionality Reduction

In [ ]:
umap_embeddings = (umap.UMAP(n_neighbors=10, n_components=20, metric='cosine', 
                                random_state=24).fit_transform(embedding_gen))

In [ ]:
print(type(umap_embeddings),'\n',umap_embeddings.shape,'\n',umap_embeddings)
print()

# HDBScan

In [ ]:
clusters = hdbscan.HDBSCAN(min_cluster_size = 3,
                               metric='euclidean', 
                               cluster_selection_method='eom').fit(umap_embeddings)
# hdbscan.HDBSCAN(algorithm='best', alpha=1.0, approx_min_span_tree=True,
#                 gen_min_span_tree=False, leaf_size=40,metric='euclidean',
#                 min_cluster_size=5, min_samples=None, p=None)
print(clusters.labels_,len(clusters.labels_))
print(np.unique(clusters.labels_),'\n',len(np.unique(clusters.labels_)))

In [ ]:
cluster=[]
for idx in range(len(clusters.labels_)):
    if clusters.labels_[idx]==10: #taking dataponits of 10th cluster
        cluster.append(idx)

In [ ]:
news_cluster=[]
for dp in cluster:
    news_cluster.append(GoogleTranslator(source='auto', target='en').translate(data[dp][:200])) #limiting to 200 --> for easy translation
news_cluster